In [1]:
import pandas as pd
import numpy as np

In [2]:
import re

In [3]:
from pandas.testing import assert_series_equal

In [130]:
acct = pd.read_csv('FEEDVISOR_pre_daily_collection_snapshot_cleaned_pai_snapshot_c1_d2019-01-01_account.csv', dtype=object)
trgt = pd.read_csv('FEEDVISOR_pre_daily_collection_snapshot_cleaned_pai_snapshot_c1_d2019-01-01_target.csv', dtype=object)

In [182]:
big_acct = pd.read_csv('FEEDVISOR_pre_daily_collection_snapshot_pre_stitch_cleaned_account_NEW2019-01-09.csv', dtype=object)
big_trgt = pd.read_csv('FEEDVISOR_pre_daily_collection_snapshot_pre_stitch_cleaned_target_NEW2019-01-09.csv', dtype=object)

In [183]:
for df in [acct, big_acct, trgt, big_trgt]:
    print(df.shape)

(52036, 24)
(1651085, 32)
(51985, 15)
(1651085, 16)


In [184]:
big_acct = big_acct[big_acct.pai_is_deleted == '0']
big_trgt = big_trgt[big_trgt.pai_is_deleted == '0']

In [185]:
print(big_acct.shape)

(129521, 32)


In [186]:
assert (big_acct.index == big_trgt.index).all()
assert (acct.index == trgt.index).all()

ValueError: Lengths must match to compare

## Preprocessing

In [ ]:
big_acct.dropna(subset=['date'], inplace=True)
big_trgt.dropna(subset=['date'], inplace=True)
acct.dropna(subset=['date'], inplace=True)
trgt.dropna(subset=['date'], inplace=True)

In [143]:
big_acct.drop(big_acct[big_acct.date == '#NAME?'].index, inplace=True)
big_trgt.drop(big_trgt[big_trgt.date == '#NAME?'].index, inplace=True)

In [10]:
big_acct.drop(big_acct[big_acct.next_calculated_exit_point == 'Non Preventable BUT addressable'].index, inplace=True)

acct.next_calculated_exit_point = acct.next_calculated_exit_point.apply(lambda x: re.sub('-0', '-', x))

big_acct.churn_probability_date = big_acct.churn_probability_date.apply(
    lambda x: re.sub('^0', '', x) if pd.notnull(x) else x)

In [11]:
acct.churn_probability_date = pd.to_datetime(acct.churn_probability_date, format='%Y-%m-%d').astype(str)
big_acct.next_calculated_exit_point = pd.to_datetime(big_acct.next_calculated_exit_point, format='%Y-%m-%d').astype(str)
acct.next_calculated_exit_point = pd.to_datetime(acct.next_calculated_exit_point, format='%Y-%m-%d').astype(str)

In [141]:
big_acct.date = big_acct.date.str.lstrip('SDR,')
big_acct.date = big_acct.date.str.rstrip('",')
big_trgt.date = big_trgt.date.str.lstrip('SDR,')
big_trgt.date = big_trgt.date.str.rstrip('",')

In [140]:
big_acct.cancellation_reason = big_acct.cancellation_reason.str.lstrip('"')
big_acct.cancellation_reason = big_acct.cancellation_reason.str.rstrip('"')

In [ ]:
big_acct.churn_probability_date = pd.to_datetime(big_acct.churn_probability_date, format='%m/%d/%Y').astype(str)

## Set/Unset time

In [187]:
acct.date = pd.to_datetime(acct.date, format='%Y-%m-%d').astype(str)
trgt.date = pd.to_datetime(trgt.date, format='%Y-%m-%d').astype(str)
big_acct.date = pd.to_datetime(big_acct.date, format='%Y-%m-%d').astype(str)
big_trgt.date = pd.to_datetime(big_trgt.date, format='%Y-%m-%d').astype(str)

AttributeError: 'DataFrame' object has no attribute 'date'

In [19]:
acct.date = pd.to_datetime(acct.date, format='%Y-%m-%d')
big_acct.date = pd.to_datetime(big_acct.date, format='%Y-%m-%d')
trgt.date = pd.to_datetime(trgt.date, format='%Y-%m-%d')
big_trgt.date = pd.to_datetime(big_trgt.date, format='%Y-%m-%d')

In [188]:
# preprocessing!
trgt.churn_in_stage = trgt.churn_in_stage.astype(float)
trgt.churn_in_stage_paying = trgt.churn_in_stage_paying.astype(int)
trgt.churn_in_stage_poc = trgt.churn_in_stage_poc.astype(float)
trgt.reached_poc_stage = trgt.reached_poc_stage.astype(int)
trgt.repricing_activated = trgt.repricing_activated.astype(int)
trgt.pai_is_deleted = trgt.pai_is_deleted.astype(int)
trgt.account_birth_date = pd.to_datetime(trgt.account_birth_date, format='%Y-%m-%d').astype(str)
big_trgt.account_birth_date = pd.to_datetime(big_trgt.account_birth_date, format='%Y-%m-%d').astype(str)
trgt.churn_month = pd.to_datetime(trgt.churn_month, format='%Y-%m-%d').astype(str)
big_trgt.churn_month = pd.to_datetime(big_trgt.churn_month, format='%Y-%m-%d').astype(str)
trgt.last_activity = pd.to_datetime(trgt.last_activity, format='%Y-%m-%d').astype(str)
big_trgt.last_activity = pd.to_datetime(big_trgt.last_activity, format='%Y-%m-%d').astype(str)
trgt.poc_end_date = pd.to_datetime(trgt.poc_end_date, format='%Y-%m-%d').astype(str)
big_trgt.poc_end_date = pd.to_datetime(big_trgt.poc_end_date, format='%Y-%m-%d').astype(str)

/Users/joshfriedlander/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [189]:
for col in ['account_currency_code', 'base_mrr', 'last_bill_total_fee', 'last_bill_variable_fee', 
 'of_repricing_online_stores']:
    acct[col] = acct[col].astype(float)

In [190]:
for col in ['account_currency_code', 'base_mrr', 'last_bill_total_fee', 'last_bill_variable_fee', 
 'of_repricing_online_stores']:
    big_acct[col] = big_acct[col].astype(float)

In [191]:
acct.contract_term = acct.contract_term.apply(lambda x: str(float(x)) if not pd.isnull(float(x)) else x)
big_acct.contract_term = big_acct.contract_term.astype(str)
acct.contract_term = acct.contract_term.apply(lambda x: str(int(float(x))) if not pd.isnull(float(x)) else x)
big_acct.contract_term = big_acct.contract_term.astype(str)
acct.country = acct.country.map({'"Korea, Republic of"':'Korea, Republic of',
                  """\"Korea, Democratic People's Republic of\"""": "Korea, Democratic People's Republic of"})

## Set and unset index

In [ ]:
acct.set_index(['pai_account_id', 'date'], inplace=True)
trgt.set_index(['pai_account_id', 'date'], inplace=True)

In [193]:
big_acct.set_index(['pai_account_id', 'date'], inplace=True)
big_trgt.set_index(['pai_account_id', 'date'], inplace=True)

In [98]:
acct.reset_index(inplace=True)
trgt.reset_index(inplace=True)
big_acct.reset_index(inplace=True)
big_trgt.reset_index(inplace=True)

## Comparison
### Account

In [238]:
spare_cols = ['client_id','recipe_execution_date']
t1 = acct[sorted(list(acct))].drop(columns=spare_cols)
t2 = big_acct[sorted(list(set(list(big_acct)) & set(list(acct))))].drop(columns=spare_cols)
joint = t1.join(t2, how='inner', rsuffix='_big')

In [239]:
n = int(joint.shape[1] / 2)
col_names = list(joint)
for (i,j) in (zip(range(n), range(n, 2 * n))):
    try:
        assert_series_equal(joint.iloc[:,i], joint.iloc[:,j], check_names=False, check_less_precise=True)
    except AssertionError as e:
        print('Prob in col {}, {}!'.format(col_names[i], col_names[j]), str(e)[:62])

Prob in col cancellation_reason, cancellation_reason_big! Series are different

Series values are different (1.83707 %)

Prob in col churn_probability_description, churn_probability_description_big! Series are different

Series values are different (7.14052 %)

Prob in col contract_term, contract_term_big! Series are different

Series values are different (0.66558 %)

Prob in col country, country_big! Series are different

Series values are different (97.73011 %)


In [240]:
joint[joint.country != joint.country_big][['country', 'country_big']].dropna()

,,country,country_big
pai_account_id,date,,


In [241]:
joint[joint.contract_term != joint.contract_term_big][['contract_term', 'contract_term_big']].dropna()

,,contract_term,contract_term_big
pai_account_id,date,,


## Differences

There are diff dates after 10/2/18 - let's remove them

In [171]:
acct.shape

(68767, 24)

In [177]:
acct = acct[acct.index.get_level_values('date') < '2018-02-10']

In [178]:
acct.shape

(52190, 24)

In [25]:
set(acct.index.get_level_values('pai_account_id')) - set(big_acct.index.get_level_values('pai_account_id'))

{'001b000003grK1H', '001b000003nDphU'}

Blacklist issue - let's drop them

In [179]:
acct = acct[~acct.index.get_level_values('pai_account_id').isin(['001b000003grK1H', '001b000003nDphU'])]

In [180]:
print(acct.shape)

(52036, 24)


In [36]:
# diff_tuples = set(big_acct.index) - set(acct.index)
diff_id = set(big_acct.index.get_level_values('pai_account_id')) - set(acct.index.get_level_values('pai_account_id'))
diff_date = set(big_acct.index.get_level_values('date')) - set(acct.index.get_level_values('date'))

Diff pai_ids are something known, ignore them - we'll drop them and recheck

In [196]:
print(big_acct.shape)

(129521, 30)


In [195]:
big_acct = big_acct[~big_acct.index.get_level_values('pai_account_id').isin(diff_id)]

In [197]:
big_acct = big_acct[~big_acct.index.get_level_values('date').isin(diff_date)]

In [198]:
print(big_acct.shape)

(129521, 30)


In [63]:
set(acct.index.get_level_values('date')) - set(big_acct.index.get_level_values('date'))

set()

In [64]:
set(big_acct.index.get_level_values('pai_account_id')) - set(acct.index.get_level_values('pai_account_id'))

set()

In [65]:
set(big_acct.index.get_level_values('date')) - set(acct.index.get_level_values('date'))

set()

In [45]:
set(acct.index.get_level_values('pai_account_id')) - set(big_acct.index.get_level_values('pai_account_id'))

set()

In [93]:
(set(big_acct.index) - set(acct.index)) == (set(big_trgt.index) - set(trgt.index))

True

In [94]:
(set(acct.index) - set(big_acct.index)) == (set(trgt.index) - set(big_trgt.index))

True

In [100]:
set(trgt.index) - set(big_trgt.index)

{('001b000003NHFmc', Timestamp('2017-10-04 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-05 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-08 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-09 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-11 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-12 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-15 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-16 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-17 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-18 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-22 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-24 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-10-25 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-11-01 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-11-04 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-11-05 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-11-07 00:00:00')),
 ('001b000003NHFmc', Timestamp('2017-11-08 00:00

In [124]:
big_acct = big_acct.drop(set(big_acct.index) - set(acct.index))

In [125]:
acct = acct.drop(set(acct.index) - set(big_acct.index))

In [114]:
l1 = big_trgt.loc[list(set(big_trgt.index) - set(trgt.index))]['account_birth_date'].tolist()
l2 = big_trgt.loc[list(set(big_trgt.index) - set(trgt.index))]['churn_month'].tolist()

In [115]:
set(list(zip(l1,l2)))

{('2013-06-09', '2016-07-01'),
 ('2013-07-10', '2016-09-01'),
 ('2014-05-12', '2014-10-01')}

In [116]:
np.unique(big_trgt.loc[list(set(big_trgt.index) - set(trgt.index))][['account_birth_date', 'churn_month']].values)

array(['2013-06-09', '2013-07-10', '2014-05-12', '2014-10-01',
       '2016-07-01', '2016-09-01'], dtype=object)

# Target

Let's apply the same changes we did to acct:

In [207]:
df.loc[~(df.pai_account_id == '001b00000054P54') & ~(df.pai_is_deleted == '1')]

,repricing_activated,client_id,pai_account_id,recipe_execution_date,pai_is_deleted,last_activity,poc_end_date,churn_month,profile,churn_in_stage_paying,account_birth_date,date,churn_in_stage_poc,reached_poc_stage,stage,churn_in_stage
6,0,1,001b000003iVRHQ,2019-01-09,0,2017-09-11,2017-08-06,2017-08-06,smb,0,2017-4-25,2017-11-08,1,1,poc,1
7,0,1,001b00000054ILE,2019-01-09,0,2017-10-16,2013-06-26,NaN,NaN,1,2013-3-26,2017-11-08,NaN,1,paying,1
13,0,1,001b00000053SKj,2019-01-09,0,2017-8-22,2013-06-27,2015-06-01,NaN,1,2013-3-27,2017-11-08,NaN,1,paying,1
17,0,1,0010J00001opUHM,2019-01-09,0,2017-11-03,2018-02-03,NaN,enterprise,0,2017-11-03,2017-11-08,0,1,poc,0
19,0,1,001b00000053Qkh,2019-01-09,0,2016-09-01,2013-05-12,2016-09-01,smb,1,2013-02-12,2017-11-08,NaN,1,paying,1
25,0,1,001b00000053HT8,2019-01-09,0,2017-08-09,2013-06-03,NaN,NaN,1,2013-03-03,2017-11-08,NaN,1,paying,1
31,0,1,001b0000004h75H,2019-01-09,0,2016-02-01,2013-08-28,2016-02-01,smb,1,2013-5-28,2017-11-08,NaN,1,paying,1
37,0,1,001b0000004c0kw,2019-01-09,0,2017-9-26,2013-06-19,NaN,NaN,1,2013-3-19,2017-11-08,NaN,1,paying,1
41,0,1,0010J00001opSZk,2019-01-09,0,2017-11-06,2018-4-28,NaN,enterprise,0,2017-10-30,2017-11-08,0,1,poc,0
67,0,1,001b0000004T2I2,2019-01-09,0,2017-08-02,2013-06-13,2015-10-01,smb,1,2013-3-13,2017-11-08,NaN,1,paying,1


In [206]:
df

,repricing_activated,client_id,pai_account_id,recipe_execution_date,pai_is_deleted,last_activity,poc_end_date,churn_month,profile,churn_in_stage_paying,account_birth_date,date,churn_in_stage_poc,reached_poc_stage,stage,churn_in_stage
0,0,1,001b000003iVRHf,2019-01-09,1,2017-4-19,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0
1,1,1,001b00000054P54,2019-01-09,0,2017-11-07,2016-04-01,NaN,enterprise,0,2013-04-10,2017-11-08,0,1,paying,0
2,0,1,001b000003oHTZN,2019-01-09,1,2017-10-10,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0
3,0,1,001b000003mX1zw,2019-01-09,1,2017-8-28,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0
4,0,1,001b000003ngOd2,2019-01-09,1,2017-7-13,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0
5,0,1,0010J00001ooX3W,2019-01-09,1,2017-10-03,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0
6,0,1,001b000003iVRHQ,2019-01-09,0,2017-09-11,2017-08-06,2017-08-06,smb,0,2017-4-25,2017-11-08,1,1,poc,1
7,0,1,001b00000054ILE,2019-01-09,0,2017-10-16,2013-06-26,NaN,NaN,1,2013-3-26,2017-11-08,NaN,1,paying,1
8,0,1,001b000003oHS31,2019-01-09,1,2017-7-31,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0
9,0,1,001b000003mX1QQ,2019-01-09,1,2017-10-19,NaN,NaN,NaN,0,NaN,2017-11-08,0,1,poc,0


In [151]:
trgt.shape

(68767, 15)

In [152]:
trgt = trgt[trgt.index.get_level_values('date') < '2018-02-10']

In [166]:
trgt.shape

(52190, 15)

In [167]:
trgt = trgt[~trgt.index.get_level_values('pai_account_id').isin(['001b000003grK1H', '001b000003nDphU'])]

In [168]:
trgt.shape

(52036, 15)

In [ ]:
big_trgt.shape

In [60]:
big_trgt = big_trgt[~big_trgt.index.get_level_values('pai_account_id').isin(diff_id)]

In [61]:
big_trgt = big_trgt[~big_trgt.index.get_level_values('date').isin(diff_date)]

In [62]:
big_trgt.shape

(96261, 14)

In [ ]:
diff_id_targ = set(big_trgt.index.get_level_values('pai_account_id')) - set(trgt.index.get_level_values('pai_account_id'))
to_check = big_trgt[big_trgt.index.get_level_values('pai_account_id').isin(diff_id_targ)].copy()
to_check.account_birth_date = pd.to_datetime(to_check.account_birth_date, format='%Y-%m-%d')
to_check.churn_month = pd.to_datetime(to_check.churn_month, format='%Y-%m-%d')
to_check[(to_check.account_birth_date > pd.to_datetime('2015-06-01')) & (to_check.churn_month > pd.to_datetime('2016-01-01'))]

In [89]:
big_trgt = big_trgt[~big_trgt.index.get_level_values('pai_account_id').isin(diff_id_targ)]

In [66]:
set(trgt.index.get_level_values('date')) - set(big_trgt.index.get_level_values('date'))

set()

In [90]:
set(big_trgt.index.get_level_values('pai_account_id')) - set(trgt.index.get_level_values('pai_account_id'))

set()

In [68]:
set(big_trgt.index.get_level_values('date')) - set(trgt.index.get_level_values('date'))

set()

In [69]:
set(trgt.index.get_level_values('pai_account_id')) - set(big_trgt.index.get_level_values('pai_account_id'))

set()

In [ ]:
big_trgt = big_trgt.drop(set(big_trgt.index) - set(trgt.index))

In [169]:
trgt = trgt.drop(set(trgt.index) - set(big_trgt.index))

In [170]:
trgt.shape

(51985, 15)

In [122]:
set(trgt.index) - set(big_trgt.index)

set()

In [123]:
set(big_trgt.index) - set(trgt.index)

set()

In [128]:
for df in [acct, big_acct, trgt, big_trgt]:
    print(df.shape)

(51985, 24)
(51985, 30)
(51985, 15)
(51985, 14)


Woohoo!

In [39]:
idx = pd.IndexSlice
big_acct.loc[idx[:, '2017-10-03'], :].head(1)

UnsortedIndexError: 'MultiIndex slicing requires the index to be lexsorted: slicing on levels [1], lexsort depth 0'

In [58]:
big_acct.xs('001b0000001x39f')

,client_id,recipe_execution_date,account_name,country,of_repricing_online_stores,account_currency_code,base_mrr,email,cancellation_reason,churn_preventability,...,profit_trend,time_zone,totango_account_health,upsell_opportunity_package,pai_is_deleted,x2nd_repricer_of_products_repriced,fba_sales_last_30_days,monthly_variable_cap,needed_touchpoints,subscription_suspended
date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2018-02-08,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2018-01-28,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2018-02-06,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2017-10-22,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2018-01-05,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2017-11-16,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2017-11-18,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0
2018-01-24,1,2019-01-09,SaveMoneyOnThis,United Kingdom,0.0,2.0,NaN,pmlangridge@gmail.com,Client is running their Amazon business down a...,Non Preventable,...,NaN,GMT,NaN,Standard Package,0,NaN,NaN,NaN,No Touchpoint Needed,0


In [246]:
acct['id'] = acct['pai_account_id'] + acct['date'].astype(str)
trgt['id'] = trgt['pai_account_id'] + trgt['date'].astype(str)

big_acct['id'] = big_acct['pai_account_id'] + big_acct['date'].astype(str)
big_trgt['id'] = big_trgt['pai_account_id'] + big_trgt['date'].astype(str)